In [44]:
import cv2
import numpy as np
from scipy.fftpack import fft, ifft, fftfreq
import os

In [45]:
def build_laplacian_pyramid(frame, levels):
    pyramid = [frame.astype(np.float32)]
    for _ in range(levels):
        frame = cv2.pyrDown(frame)
        pyramid.append(frame.astype(np.float32))
    return pyramid

In [46]:
def reconstruct_from_laplacian_pyramid(pyramid):
    frame = pyramid[-1]
    for level in reversed(pyramid[:-1]):
        frame = cv2.pyrUp(frame, dstsize=(level.shape[1], level.shape[0]))
        frame = cv2.add(frame, level)
    return np.clip(frame, 0, 255).astype(np.uint8)

In [47]:
def temporal_bandpass_filter(frames, freq_min, freq_max, fps):
    fft_result = fft(frames, axis=0)
    frequencies = fftfreq(frames.shape[0], d=1.0/fps)
    mask = (np.abs(frequencies) >= freq_min) & (np.abs(frequencies) <= freq_max)
    fft_result[~mask] = 0
    filtered = np.real(ifft(fft_result, axis=0))
    return filtered

In [48]:
def evm(video_path, output_path, freq_min=0.5, freq_max=1.0, amplification=20, levels=3, fps=30):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps_in = cap.get(cv2.CAP_PROP_FPS)
    
    if fps is None:
        fps = fps_in

    print(f"Input FPS: {fps_in}, Using FPS: {fps}")
    print(f"Frames: {frame_count}, Resolution: {width}x{height}")

    # Step 1: Load frames and build pyramids
    pyramids = []
    for _ in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pyramid = build_laplacian_pyramid(frame, levels)
        pyramids.append(pyramid)
    cap.release()

    # Step 2: Convert pyramid level to array
    level_frames = np.array([pyramids[i][levels] for i in range(len(pyramids))])
    
    # Step 3: Temporal filtering
    filtered = temporal_bandpass_filter(level_frames, freq_min, freq_max, fps)
    amplified = filtered * amplification

    # Step 4: Add back to pyramid and reconstruct
    out_frames = []
    for i in range(len(pyramids)):
        lap_pyr = pyramids[i]
        lap_pyr[levels] += amplified[i]
        recon = reconstruct_from_laplacian_pyramid(lap_pyr)
        out_frames.append(recon)

    # Step 5: Save output video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    for frame in out_frames:
        bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        out.write(bgr)
    out.release()
    print(f"EVM video saved to {output_path}")

In [49]:
evm(
    video_path='merged_preprocessed_videos/Unbalance_weight/front.avi',
    output_path='evm_videos/Unbalance_weight/front_evm.avi',
    freq_min=0.5,
    freq_max=1.0,
    amplification=30,
    levels=3,
    fps=30  # match this to your synced video FPS
)
evm(
    video_path='merged_preprocessed_videos/Unbalance_weight/angle.avi', 
    output_path='evm_videos/Unbalance_weight/angle_evm.avi',

    freq_min=0.5,
    freq_max=1.0,
    amplification=30,
    levels=3,
    fps=30  # match this to your synced video FPS
)
evm(
    video_path='merged_preprocessed_videos/Bearing_fault/front.avi', 
    output_path='evm_videos/Bearing_fault/front_evm.avi',
    freq_min=0.5,
    freq_max=1.0,
    amplification=30,
    levels=3,
    fps=30  # match this to your synced video FPS
)
evm(
    video_path='merged_preprocessed_videos/Bearing_fault/angle.avi', 
    output_path='evm_videos/Bearing_fault/angle_evm.avi',
    freq_min=0.5,
    freq_max=1.0,
    amplification=30,
    levels=3,
    fps=30  # match this to your synced video FPS
)
evm(
    video_path='merged_preprocessed_videos/Normal_state/front.avi', 
    output_path='evm_videos/Normal_state/front_evm.avi',
    freq_min=0.5,
    freq_max=1.0,
    amplification=30,
    levels=3,
    fps=30  # match this to your synced video FPS
)
evm(
    video_path='merged_preprocessed_videos/Normal_state/angle.avi',
    output_path='evm_videos/Normal_state/angle_evm.avi',
    freq_min=0.5,
    freq_max=1.0,
    amplification=30,
    levels=3,
    fps=30  # match this to your synced video FPS
)

Input FPS: 30.0, Using FPS: 30
Frames: 2819, Resolution: 324x132
EVM video saved to evm_videos/Unbalance_weight/front_evm.avi
Input FPS: 30.0, Using FPS: 30
Frames: 2819, Resolution: 324x132
EVM video saved to evm_videos/Unbalance_weight/angle_evm.avi
Input FPS: 30.0, Using FPS: 30
Frames: 2819, Resolution: 324x132
EVM video saved to evm_videos/Bearing_fault/front_evm.avi
Input FPS: 30.0, Using FPS: 30
Frames: 2819, Resolution: 324x132
EVM video saved to evm_videos/Bearing_fault/angle_evm.avi
Input FPS: 30.0, Using FPS: 30
Frames: 2819, Resolution: 324x132
EVM video saved to evm_videos/Normal_state/front_evm.avi
Input FPS: 30.0, Using FPS: 30
Frames: 2819, Resolution: 324x132
EVM video saved to evm_videos/Normal_state/angle_evm.avi
